# WEEK 1

Initialising the "matters" and their corresponding values.

In [48]:
from __future__ import annotations
from sympy import symbols, Eq, solve, sympify
from typing import Optional, Dict
g = 9.81

In [49]:
class matter:
    def __init__(self):
        self.mass: float | None = None
        self.volume: float | None = None
        self.density: float | None = None
        self.state: str | None = None
        self.weight: float | None = None
        self.unit_weight: float | None = None

As a 3 state matters, we can define Air, Soil and Water to be child classes of matter, and they will inherit all the methods

In [50]:
class air(matter):
    def __init__(self):
        super().__init__()
        self.state = "gas"
        self.density = 0
class solid(matter):
    def __init__(self):
        super().__init__()
        self.state = "solid"
        self.density: float | None = None
        self.specific_gravity: float | None = None
class liquid(matter):
    def __init__(self):
        super().__init__()
        self.state = "liquid"
        self.density: float | None = None

Now, we can define a soil sample, as a class that encapsulate the previous 3 classes
Here, I choose to define variables as functions, and solve them using **NUMERICAL METHOD**

In [51]:
class soil:
    def __init__(self):
        self.air = air()
        self.solid = solid()
        self.water = liquid()
        self.total_mass:float|None = None
        self.total_volume:float|None = None
        self.porosity:float|None = None
        self.moisture_content:float|None = None
        self.saturation:float|None = None
        self.relative_density:float|None = None
        self.void_ratio:float|None = None
        self.solid_fraction:float|None = None
        self.dry_unit_weight:float|None = None
        self.bulk_unit_weight:float|None = None
        self.saturated_unit_weight:float|None = None
        self.submerged_unit_weight:float|None = None
        self.degree_of_compaction:float|None = None
        self.e_max:float|None = None
        self.e_min:float|None = None
        if self.water.density is None:
            self.water.density = 998
    def solve(self) -> Dict[str, float]:
        # --- 1) Symbols
        Va, Vw, Vs, Vv, Vt = symbols('Va Vw Vs Vv Vt', real=True)
        Gs,e, n, S, v, Id, e_max, e_min = symbols('Gs e n S v Id e_max e_min', real=True)
        Mw, Ms, Mc = symbols('Mw Ms Mc', real=True)
        gamma_water, gamma_bulk, gamma_sat, gamma_dry, gamma_sub = symbols('gamma_water gamma_bulk gamma_sat gamma_dry gamma_sub', real=True)
        Wsolid, Wwater = symbols('Wsolid Wwater', real=True)
        # --- 2) Equation set (extend as you add more relations)
        # Volumes
        eqs = [
            
            #Mass Formula
            Eq(Mw, Vw*self.water.density),
            Eq(Ms, Vs*Gs*self.water.density),
            #Weights formula
            Eq(Wwater, Vs*gamma_water),
            Eq(Wsolid, Vs*Gs*gamma_water),
            #Volumes formula
            Eq(Vv, Va + Vw),       # void volume
            Eq(Vt, Vv + Vs),       # total volume
            
            # Classic soil relations
            Eq(e,  Vv / Vs),       # void ratio
            Eq(n,  Vv / Vt),       # porosity
            Eq(v, Vs / Vt),       # solid fraction
            Eq(Id, (e_max - e) / (e_max - e_min)),  # relative density (need e_max, e_min)

            # Moisture content (mass-based)
            Eq(Mc,  Mw / Ms),
            Eq(S,  Vw / Vv),       # degree of saturation
            
            #Unit Weights
            Eq(gamma_bulk, (Mw + Ms) / Vt * g),
            Eq(gamma_dry, Ms / Vt * g),
            Eq(gamma_sat, (Ms)* g / (Vs+Vw)),
            Eq(gamma_sub, gamma_sat - g * self.water.density),
            Eq(gamma_water, self.water.density*g)
        ]

        # --- 3) Gather knowns from the instance
        known: Dict = {}

        def put(symbol, value):
            if value is not None:
                known[symbol] = float(value)
        put(Va, getattr(self.air,   'volume', None))
        put(Vw, getattr(self.water, 'volume', None))
        put(Vs, getattr(self.solid, 'volume', None))
        put(Mw, getattr(self.water, 'mass',   None))
        put(Ms, getattr(self.solid, 'mass',   None))
        put(Gs, getattr(self.solid, 'specific_gravity', None))
        put(e_max, self.e_max)
        put(e_min, self.e_min)
        put(gamma_bulk,     self.bulk_unit_weight)
        put(gamma_water,    self.water.unit_weight)
        put(gamma_dry,      self.dry_unit_weight)
        put(gamma_sat,      self.saturated_unit_weight)
        put(gamma_sub,      self.submerged_unit_weight)
        put(e,  self.void_ratio)
        put(n,  self.porosity)
        put(S,  self.saturation)
        put(v,  self.solid_fraction)
        put(Mc,  self.moisture_content)
        put(Vt, self.total_volume)

        # --- 4) Helper: single-unknown solve loop
        def single_unknown_pass(equations, known_dict):
            updated = False
            for eq in equations:
                # substitute knowns
                residual = (eq.lhs - eq.rhs).subs(known_dict)
                # which symbols remain?
                unknown_syms = [s for s in residual.free_symbols if s not in known_dict]
                if len(unknown_syms) == 1:
                    u = unknown_syms[0]
                    # solve for that symbol
                    sol = solve(sympify(residual), u)
                    if sol:
                        val = float(sol[0])
                        known_dict[u] = val
                        updated = True
            return updated

        # Keep solving until no change
        while single_unknown_pass(eqs, known):
            pass

        # --- 5) Write back to the instance (only if previously None)
        def set_if_none(attr_name, sym):
            val = known.get(sym, None)
            if val is not None and getattr(self, attr_name) is None:
                setattr(self, attr_name, val)

        def set_obj_if_none(obj, field, sym):
            val = known.get(sym, None)
            if val is not None and getattr(obj, field) is None:
                setattr(obj, field, val)

        set_obj_if_none(self.air,   'volume', Va)
        set_obj_if_none(self.water, 'volume', Vw)
        set_obj_if_none(self.solid, 'volume', Vs)

        set_if_none('total_volume', Vt)
        set_if_none('void_ratio',   e)
        set_if_none('porosity',     n)
        set_if_none('saturation',   S)
        set_if_none('solid_fraction', v)
        set_if_none('moisture_content', Mc)

        # Return what we learned (nice for debugging)
        # You can trim this if you only want final state on the object
        out = {}
        for name, sym in {
            'Va': Va, 'Vw': Vw, 'Vs': Vs, 'Vv': Vv, 'Vt': Vt,
            'e': e, 'n': n, 'S': S, 'v': v, 'Id': Id, 'Mw': Mw, 'Ms': Ms
        }.items():
            if sym in known:
                out[name] = known[sym]

        return out


Here, you just simply input all known variables, and the software will automatically solves for ALL possible missing variables

In [53]:
s = soil()
s.solid.volume = 50   # m^3
s.moisture_content = 0.1
s.air.volume = 0.06
# s.water.density = 998
s.solid.specific_gravity = 2.65
results = s.solve()
print(s.water.volume)
print(results)

13.25
{'Va': 0.06, 'Vw': 13.25, 'Vs': 50.0, 'Vv': 13.31, 'Vt': 63.31, 'e': 0.2662, 'n': 0.21023534986574, 'S': 0.99549211119459, 'v': 0.78976465013426, 'Mw': 13223.499999999995, 'Ms': 132235.0}
